### Conversational AI (Chatbot)

In [14]:
import os
from dotenv import load_dotenv
import gradio as gr
import google.generativeai as genai
from IPython.display import Markdown, display, clear_output

In [15]:
load_dotenv()

google_api_key = os.getenv('GEMINI_API_KEY')

In [16]:
genai.configure(api_key=google_api_key)

In [17]:
system_message = "You are a helpful assistant"

In [18]:
def message_gemini(prompt):
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",  # or "gemini-1.5-pro"
        system_instruction=system_message
    )
    
    response = model.generate_content(prompt)
    return response.text


In [19]:
view = gr.Interface(
    fn=message_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


### Streaming Output

In [20]:

system_message = "You are a helpful assistant that responds in markdown"  # Optional system instruction

def stream_gemini(prompt):
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message
    )

    stream = model.generate_content(prompt, stream=True)

    result = ""
    for chunk in stream:
        if chunk.text:
            result += chunk.text
            yield result


In [21]:
for partial in stream_gemini("Explain quantum computing in simple terms."):
    clear_output(wait=True)
    display(Markdown(partial))

Okay, let's break down quantum computing in a way that's (hopefully) easy to understand:

**The Core Idea: It's a different way to calculate things.**

Think of regular computers (like your phone or laptop) as light switches:

*   **Regular Computers (Classical Computers):**  A light switch can be either ON (representing a 1) or OFF (representing a 0).  These 0s and 1s are called **bits**.  All calculations are done using these bits that are either 0 or 1.  It's like following a set of rules using only "yes" or "no" answers.

*   **Quantum Computers:** Now imagine a dimmer switch.  It can be ON, OFF, or *anywhere in between*.  Think of it like a dial that can be at 0, 1, or a value somewhere in between. This "in-between" state is the key. Instead of bits, quantum computers use **qubits** (quantum bits). A qubit can be a 0, a 1, or **both 0 and 1 *at the same time***. This is called **superposition**.

**Key Quantum Concepts Explained:**

1.  **Superposition:** Imagine flipping a coin. Before it lands, it's *kind of* both heads and tails at the same time. A qubit is like that coin spinning in the air. It's in a combination of 0 and 1 until we "look" at it (measure it), at which point it "collapses" into either a 0 or a 1.  This ability to be in multiple states simultaneously allows quantum computers to explore many possibilities at once.

2.  **Entanglement:** Imagine two of our special coins.  These coins are linked in a spooky way. If you flip one and it lands on heads, you *instantly* know the other one will land on tails (even if they are miles apart!).  This is entanglement.  Entangled qubits are linked together, and their fates are intertwined.  Measuring one entangled qubit instantly tells you something about the other, no matter the distance. This allows quantum computers to perform calculations in a coordinated way.

**So, what does this *mean*?**

Because qubits can be in multiple states at once (superposition) and can be linked together (entanglement), quantum computers can:

*   **Explore many possibilities simultaneously:**  Instead of trying options one at a time (like a regular computer), a quantum computer can explore *all* the options at the same time.  This is like searching a maze by splitting yourself into many copies and exploring every path at once.
*   **Solve certain problems much faster:** For specific types of problems, this "parallel processing" can lead to a massive speedup compared to regular computers.

**What are Quantum Computers good at?**

Quantum computers are *not* going to replace your laptop. They're good for very specific types of problems that are incredibly difficult for regular computers, such as:

*   **Drug discovery and materials science:** Simulating molecules to design new drugs or materials.
*   **Cryptography:** Breaking existing encryption algorithms (and creating new, quantum-resistant ones).
*   **Optimization:** Finding the best solution from a vast number of possibilities (e.g., optimizing traffic flow, financial modeling, logistics).
*   **Machine Learning:** Developing new and more powerful machine learning algorithms.

**Why aren't we all using quantum computers already?**

*   **They are very difficult to build:** Qubits are extremely fragile and easily disturbed by the environment (heat, vibrations, etc.). Maintaining their quantum state is a huge technological challenge.
*   **They are expensive:** Building and maintaining quantum computers requires specialized equipment and expertise.
*   **They are still in early stages of development:**  The field is rapidly advancing, but quantum computers are not yet ready for widespread use.  The number of qubits is relatively small, and they are prone to errors.
*   **Programming is hard:**  You can't just run your regular programs on a quantum computer.  You need to write special quantum algorithms.

**In a Nutshell:**

Quantum computing uses the weird laws of quantum mechanics to perform calculations in a fundamentally different way than regular computers.  This allows them to potentially solve certain problems much faster, but they are still in early stages of development and are not a replacement for your everyday computer. They're more like specialized tools for specific, very complex problems.


In [22]:
view = gr.Interface(
    fn=stream_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=gr.Markdown(),
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


### Chat Bot With History

In [23]:
def stream_gemini_with_history(message, history):
        
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message
    )
    
    prompt = ""
    for user_msg, bot_msg in history:
        prompt += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    prompt += f"User: {message}\nAssistant:"

    stream = model.generate_content(prompt, stream=True)

    response = ""
    for chunk in stream:
        if chunk.text:
            response += chunk.text
            yield history + [(message, response)], ""


In [24]:
def clear_chat():
    return [], ""

with gr.Blocks(theme=gr.themes.Soft()) as view:
    gr.Markdown("## 💬 Chat Bot with History")

    chatbot = gr.Chatbot(label="Conversation", show_copy_button=True)
    msg = gr.Textbox(label="Your message:", lines=2, placeholder="Type your message here...")

    with gr.Row():
        send_btn = gr.Button("Send")
        clear_btn = gr.Button("Clear Chat")

    msg.submit(stream_gemini_with_history, inputs=[msg, chatbot], outputs=[chatbot, msg])
    send_btn.click(stream_gemini_with_history, inputs=[msg, chatbot], outputs=[chatbot, msg])

    clear_btn.click(clear_chat, outputs=[chatbot, msg])

view.launch()


C:\Users\chandula\AppData\Local\Temp\ipykernel_15184\1344445376.py:7: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", show_copy_button=True)


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
